In [1]:
# Title: CNN CIFAR10 dataset
# Author: Izza Claire M. Jalandoni
# Created: March 2, 2019 using Google Colab

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers import Dense, Dropout, Input, Activation
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.utils import to_categorical, plot_model
from keras.utils.data_utils import Sequence
from keras.datasets import cifar10

(x_train,y_train),(x_test,y_test) = cifar10.load_data() # loading datasets
num_labels = len(np.unique(y_train))                    # computing number labels
num_samples = len(x_train)

#print("x_train shape:", x_train.shape)
#print(x_train.shape[0],"train samples")
#print("Number of labels: %d" % num_labels)

y_train = to_categorical(y_train,num_labels)
y_test = to_categorical(y_test,num_labels)

x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255

# set hyperparameters
data_augmentation = True
batch_size = 128
kernel_size = 3
pool_size = 2
filters = 64
dropout = 0.2
epochs = 50

steps_per_epoch = num_samples // batch_size

# Observations:
# 1. Adam best optimizer but converges fast, increasing epochs only encourage overfitting
# 2. Adding dropout in other layers decreases accuracy
# 3. Trying to add batch normalization, adding increased to 79%

# using sequential to build network
model = Sequential()
model.add(Conv2D(filters, (kernel_size, kernel_size),
                activation='relu',input_shape=x_train.shape[1:]))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size))
model.add(Conv2D(filters, kernel_size=kernel_size,
                activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size))
model.add(Conv2D(filters, kernel_size=kernel_size,
                activation='relu'))
model.add(Flatten())
model.add(Dropout(dropout))
model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()

plot_model(model,to_file='cnn-cifar.png',show_shapes=True)

model.compile(loss='categorical_crossentropy', optimizer='adam',
             metrics=['accuracy'])

if data_augmentation:
  print("Augmenting data.")
  datagen = ImageDataGenerator(
    featurewise_center = False, # set input mean to 0 over dataset
    samplewise_center = False,  # set each sample mean to 0
    featurewise_std_normalization = False,
    samplewise_std_normalization = False,
    zca_whitening = False,
    zca_epsilon = 1e-06,
    rotation_range = 0,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    shear_range = 0,
    zoom_range = 0,
    channel_shift_range = 0,
    fill_mode = 'nearest',
    cval = 0,
    horizontal_flip = True,
    vertical_flip = True,
    rescale = None,
    preprocessing_function = None,
    data_format = None,
    validation_split = 0.0)

  datagen.fit(x_train)
  history = model.fit_generator(datagen.flow(x_train,y_train,batch_size=batch_size),
                      epochs=epochs, steps_per_epoch=steps_per_epoch,
                      validation_data=(x_test,y_test), workers=4)

else:
  print("Not augmenting data.")
  history = model.fit(x_train,y_train,epochs=epochs,validation_split=0.3,
                      batch_size=batch_size)

loss,acc = model.evaluate(x_test,y_test,batch_size=batch_size)
print("Test Accuracy: %.lf%%" % (100.0*acc))

# priting history
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','test'],loc='upper left')
plt.show()


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 64)        1792      
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 30, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 64)        36928     
_________________________________________________________________
batch_normalization_2 (Batch (None, 13, 13, 64)        256       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 64)          36928     
__________

390/390 [==============================] - 11s 28ms/step - loss: 0.7345 - acc: 0.7435 - val_loss: 0.7689 - val_acc: 0.7378
Epoch 49/50
390/390 [==============================] - 11s 28ms/step - loss: 0.7316 - acc: 0.7448 - val_loss: 0.7888 - val_acc: 0.7349
Epoch 50/50
10000/10000 [==============================] - 0s 48us/step
Test Accuracy: 74%


<Figure size 640x480 with 1 Axes>